<a href="https://colab.research.google.com/github/Mateusfcs/Data-Science/blob/main/BigData_Analise_Vendas_com_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**

## Spark UI

In [1]:
!pip install -q pyspark
!pip install -q pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
spark

In [3]:
# Fazer login no site https://dashboard.ngrok.com/get-started/setup para obter autenticação própria
ngrok_token = '27WqUGZ1SkzPl4bQunMXJyasbWZ_5pCgLCsppL7ufk9rzK7j4'

In [4]:
get_ipython().system_raw(f'ngrok authtoken {ngrok_token}')
get_ipython().system_raw('ngrok http 4050 &')
!sleep 3
print('URL para interface Spark:')
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

URL para interface Spark:


## Libraries

In [5]:
import pandas as pd
from google.colab import files

import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Load data

In [6]:
# Load file
_ = files.upload()

Saving vgsales.csv to vgsales (1).csv


## Data Description

This dataset contains records of popular video games in North America, Japan, Europe and other parts of the world. Every video game in this dataset has at least 100k global sales.

[Source](https://www.kaggle.com/datasets/gregorut/videogamesales/code?datasetId=284&sortBy=voteCount) of dataset.

In [9]:
df = spark.read.csv('vgsales.csv', header=True, inferSchema=True)

In [14]:
df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

## Data Dictionary

| Column       | Explanation                                               |
|:-------------|:----------------------------------------------------------|
| Rank         | Ranking of overall sales                                  |
| Name         | Name of the game                                          |
| Platform     | Platform of the games release (i.e. PC,PS4, etc.)         |
| Year         | Year the game was released in                             |
| Genre        | Genre of the game                                         |
| Publisher    | Publisher of the game                                     |
| NA_Sales     | Number of sales in North America (in millions)            |
| EU_Sales     | Number of sales in Europe (in millions)                   |
| JP_Sales     | Number of sales in Japan (in millions)                    |
| Other_Sales  | Number of sales in other parts of the world (in millions) |
| Global_Sales | Number of total sales (in millions)                       |

# Questões

## Questão 1

Leia os dados com Spark, certificando-se que

1. A tabela tenha duas partições e

2. As colunas da tabela tenham este respectivo schema:

|Column|Data type|
|---|---|
|Rank|integer|
|Name|string|
|Platform|string|
|Year|integer|
|Genre|string|
|Publisher|string|
|NA_Sales|double|
|EU_Sales|double|
|JP_Sales|double|
|Other_Sales|double|
|Global_Sales|double|

In [37]:
#df.printSchema()
df.dtypes

[('Rank', 'int'),
 ('Name', 'string'),
 ('Platform', 'string'),
 ('Year', 'string'),
 ('Genre', 'string'),
 ('Publisher', 'string'),
 ('NA_Sales', 'double'),
 ('EU_Sales', 'double'),
 ('JP_Sales', 'double'),
 ('Other_Sales', 'double'),
 ('Global_Sales', 'double')]

## Questão 2

Construa uma tabela com:
- contagem das observações
- média
- desvio padrão
- valor mínimo
- valor máximo

para todas as coluna de vendas (que possuem "Sales" no nome).

**Dica:** uma **única função** consegue calcular todos estes valores.

In [27]:
#Filtrando as colunas com 'Sales' no nome
sales_columns = [col for col in df.columns if 'Sales' in col]

# Calculando count, mean, std, min e max das colunas de  'Sales'
sales_summary = df.select(sales_columns).describe()

# Exibindo as informações
sales_summary.show()

+-------+-------------------+------------------+-------------------+--------------------+------------------+
|summary|           NA_Sales|          EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+-------------------+------------------+-------------------+--------------------+------------------+
|  count|              16598|             16598|              16598|               16598|             16598|
|   mean|0.26466742981084057|0.1466520062658483|0.07778166044101108|0.048063019640913515|  0.53744065550074|
| stddev| 0.8166830292988798|0.5053512312869136| 0.3092906480822022| 0.18858840291271395|1.5550279355699066|
|    min|                0.0|               0.0|                0.0|                 0.0|              0.01|
|    max|              41.49|             29.02|              10.22|               10.57|             82.74|
+-------+-------------------+------------------+-------------------+--------------------+------------------+



## Questão 3

Para cada variável categórica do tipo string, calcule quantas categorias distintas estão presentes.

In [35]:
string_columns = [col for col, dtype in df.dtypes if dtype == 'string']

# Calculando o número de categorias distintas para cada coluna
distinct_counts = {}
for col in string_columns:
    distinct_count = df.select(col).distinct().count()
    distinct_counts[col] = distinct_count

# Exibindo o número de categorias distintas para cada coluna
for col, count in distinct_counts.items():
    print(f"A coluna '{col}' possui {count} categorias distintas.")

A coluna 'Name' possui 11493 categorias distintas.
A coluna 'Platform' possui 31 categorias distintas.
A coluna 'Year' possui 40 categorias distintas.
A coluna 'Genre' possui 12 categorias distintas.
A coluna 'Publisher' possui 579 categorias distintas.


**Resposta:** A região da Europa e "Outras regiões" tiverem um aumento notável quando comparado ao ínicio da década de 90.

## Questão 4

Qual plataforma vendeu mais, mundialmente, considerando todo o período histórico diponível dos dados?  
Mostre as top 10 plataformas em vendas.

In [49]:
from pyspark.sql.functions import sum, desc
#Ordenando pela coluna de Plataforma e calculando vendas totais
platform_sales = df.groupBy("Platform").agg(sum("Global_Sales").alias("Total_Sales"))

# Ordenando em ordem decrescente e mostrando as top 10 plataformas em vendas
top_10_platforms = platform_sales.orderBy(desc("Total_Sales")).limit(10)

top_10_platforms.show()

+--------+------------------+
|Platform|       Total_Sales|
+--------+------------------+
|     PS2|1255.6399999999871|
|    X360| 979.9599999999996|
|     PS3| 957.8399999999987|
|     Wii| 926.7099999999971|
|      DS| 822.4899999999874|
|      PS|  730.659999999997|
|     GBA|  318.499999999998|
|     PSP| 296.2799999999948|
|     PS4| 278.0999999999994|
|      PC|258.81999999999846|
+--------+------------------+



## Questão 5

Faça uma tabela com os jogos que aparecem em múltiplas plataformas, ordene de forma que os jogos com mais plataformas apareçam primeiro e responda:
- Qual jogo aparece em mais plataformas? Em quantas plataformas?

In [53]:
from pyspark.sql.functions import countDistinct
# Agrupar por nome do jogo e contar o número de plataformas únicas
multi_platform_games = df.groupBy("Name").agg(countDistinct("Platform").alias("Num_Platforms"))

# Ordenar em ordem decrescente com base no número de plataformas
multi_platform_games = multi_platform_games.orderBy(desc("Num_Platforms"))

# Exibir a tabela com os jogos em múltiplas plataformas
multi_platform_games.show()




+--------------------+-------------+
|                Name|Num_Platforms|
+--------------------+-------------+
|Need for Speed: M...|           10|
|LEGO Marvel Super...|            9|
|       Madden NFL 07|            9|
|         Ratatouille|            9|
|             FIFA 14|            9|
|             FIFA 15|            8|
|            Terraria|            8|
|            Monopoly|            8|
|     LEGO The Hobbit|            8|
| LEGO Jurassic World|            8|
|       Madden NFL 08|            8|
|      FIFA Soccer 13|            8|
|Lego Batman 3: Be...|            8|
|LEGO Harry Potter...|            8|
|LEGO Star Wars II...|            8|
|Angry Birds Star ...|            8|
|                Cars|            8|
|The LEGO Movie Vi...|            8|
|      FIFA Soccer 08|            7|
|        Spider-Man 3|            7|
+--------------------+-------------+
only showing top 20 rows



**Resposta:** Need for Speed: Most Wanted. Aparece em 10 plataformas.

## Questão 6

Utilize a API do Pandas no Spark para calcular a soma das vendas globais para cada ano e gênro de jogo. Faça então um gráfico de linhas com os anos no eixo `x`, as vendas no eixo `y`, de forma que cada linha corresponda a um gênero de jogo.

**Dica:** após o cálculo, passar os dados para Pandas antes da plotagem, ou plotar diretamente aproveitando os métodos da classe `pyspark.pandas.DataFrame`.

Analise o gráfico e responda:
- Entre 1980 e 1990, quais gêneros mais venderam?
- Entre 2000 e 2015, quais gêneros mais venderam?

In [79]:

(
  df
 .pandas_api()
 .groupby(['Year','Genre'])[['Global_Sales']].sum()
 .sort_index()
 .reset_index()
 .plot.line(x='Year', y='Global_Sales', color='Genre')
 .update_layout(title='Genre global sales through time', template='plotly_white')
 )


**Resposta:**
- Entre 1980 e 1990 sobressaíram as vendas de plataforma, tiro (shooter) e puzzle.
- Entre 2000 e 2015 teve vendas elevadas de jogos de ação e esporte.

## Questão 7

Registre a tabela usando `createOrReplaceTempView` e faça uso da linguagem SQL criar uma tabela que:
- Considere apenas os anos da década de 90 e 
- Agrupe por ano para responder quantos % cada região teve do total de vendas (vendas globais).

Salve o resultado desta query em uma variável chamada `df_questao7`.

Após isso, execute o seguinte comando `df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')` e responda:
- Qual/quais regiões tiveram, relativamente (comparado às vendas globais), mais vendas no fim da década de noventa do que no início?








In [71]:
df.createOrReplaceTempView("vendas")

# Executando query SQL para calcular as vendas por região na década de 90
query = """
    SELECT
        Year,
        (SUM(NA_Sales) / SUM(Global_Sales)) * 100 AS Percent_NA_Sales,
        (SUM(EU_Sales) / SUM(Global_Sales)) * 100 AS Percent_EU_Sales,
        (SUM(JP_Sales) / SUM(Global_Sales)) * 100 AS Percent_JP_Sales,
        (SUM(Other_Sales) / SUM(Global_Sales)) * 100 AS Percent_Other_Sales
    FROM
        vendas
    WHERE
        Year >= 1990 AND Year <= 1999
    GROUP BY
        Year
    ORDER BY
        Year
"""

# Salvando o resultado em uma variável
df_questao7 = spark.sql(query)

# Utilizando API do Pandas para formatar o resultado
df_questao7_pandas = df_questao7.toPandas().set_index('Year').style.background_gradient(cmap='Oranges')

df_questao7_pandas


,Percent_NA_Sales,Percent_EU_Sales,Percent_JP_Sales,Percent_Other_Sales
Year,,,,
1990,51.548897,15.448471,30.127556,2.834582
1991,39.590444,12.255662,45.857896,2.295998
1992,44.472164,15.375525,37.959559,2.166492
1993,32.883863,10.113093,55.089169,1.935624
1994,35.556398,18.794998,42.932929,2.778830
1995,28.169334,16.910680,51.923732,2.996255
1996,43.565152,23.730856,28.842581,3.861411
1997,47.143994,24.042193,24.315852,4.542741
1998,50.048739,26.084922,19.511054,4.300698


## Questão 8

Calcule quantas vezes cada região teve vendas superiores às demais.

In [74]:
regions = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

(
  df
 .withColumn('max_sale', F.greatest(*regions))
 .withColumn(
     'greatest_sale_region',
     F.when(F.col('NA_Sales')==F.col('max_sale'), 'NA_Sales')
     .when(F.col('EU_Sales')==F.col('max_sale'), 'EU_Sales')
     .when(F.col('JP_Sales')==F.col('max_sale'), 'JP_Sales')
     .otherwise('Other_Sales')
 )
 .groupby('greatest_sale_region')
 .count()
 .show()
)

+--------------------+-----+
|greatest_sale_region|count|
+--------------------+-----+
|            EU_Sales| 2380|
|            JP_Sales| 4029|
|            NA_Sales|10113|
|         Other_Sales|   76|
+--------------------+-----+



## Questão 9

Construe uma tabela que mostre a diferença do total de vendas em um década com a década anterior e responda:
- Qual década apresentou a **menor diferença** comparada à década anterior?

In [77]:
w = Window.orderBy('Decade')

(
  df
 .where('Year IS NOT NULL')
 .withColumn('Decade', F.floor(F.col('Year')/10)*10) 
 .groupby('Decade')
 .agg(F.mean('Global_Sales').alias('global_sales_mean'))
 .withColumn('global_sales_mean_lag', F.lag('global_sales_mean').over(w))
 .withColumn('Delta_global_sales_mean', F.col('global_sales_mean') - F.col('global_sales_mean_lag'))
 .orderBy('Decade')
 .show()
)

+------+-------------------+---------------------+-----------------------+
|Decade|  global_sales_mean|global_sales_mean_lag|Delta_global_sales_mean|
+------+-------------------+---------------------+-----------------------+
|  null|0.36929889298892965|                 null|                   null|
|  1980|  1.836975609756096|  0.36929889298892965|     1.4676767167671665|
|  1990| 0.7229564725833695|    1.836975609756096|    -1.1140191371727266|
|  2000| 0.5043462206777033|   0.7229564725833695|    -0.2186102519056662|
|  2010| 0.4900000000000273|   0.5043462206777033|   -0.01434622067767...|
|  2020|               0.29|   0.4900000000000273|   -0.20000000000002732|
+------+-------------------+---------------------+-----------------------+



## Questão 10 - PLUS

Utilizando apenas a sintaxe do pyspark, faça uma função que calcule a moda do dataframe para um grupo de colunas que o usuário vai especificar. Além da moda, o dataframe de saída deve conter a quantidade de vezes que a moda aparece, e tambem se é multimodal.

Sua função deve receber como entrada:
- O pyspark dataframe
- A coluna-alvo que desejamos saber a moda e
- Os grupos (colunas do dataframe) que iremos considerar para calcular a moda 

O resultado será uma linha por grupo, contendo a identificação do grupo, a moda, a quantidade de vezes que apareceu, e se é um caso multimodal.

Por exemplo, considerando o seguinte dataframe:

```
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()
```

```
+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+
```

Ao aplicar a função:  
`calculate_mode(df=df_test, target_col='values', group_cols=['main_group','inner_group']).show()`

o retorno deve ser:

```
+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+
```

pois no grupo `A1-A` o valor 2.4 é o que mais aparece (2x), e nenhum outro valor aparece duas vezes também. Já no grupo `A1-B` o valor 2.4 também é o que mais aparece (2X), porém em conjunto com o 2.5, que também aparece duas 2X. Neste caso, devemos reportar apenas uma das modas e informar que é multimodal.

In [80]:
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()

+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+



In [81]:
def calculate_mode(df, target_col, group_cols=None):
  """Calculate mode

  Parameters
  ----------
  df : pyspark.DataFrame
    The dataframe in which the function will be applied
  target_col : str
    Column name in which the mode will be calculated from
  group_cols : list
    List column names to make the group. If None, no grouping will be considered.

  Returns
  -------
  output: pyspark.DataFrame
    Dataframe with group identification (one per line), the mode, value counts,
    and a column specifying if the group has more than one mode.
  """

  if group_cols is None:
    window_vals = Window.partitionBy('count')
    df_mode = df.groupby(target_col).count()
    max_count = df_mode.select(F.max('count')).collect()[0][0]

    df_mode = df_mode.filter(F.col('count')==max_count)
    nrows = df_mode.count()

    df_mode = df_mode.select(F.col(target_col).alias('mode'), F.col('count').alias('mode_counts'), (F.lit(nrows)>1).alias('multimodal'))

  else:
    window_grp = Window.partitionBy(*group_cols)
    window_grp_vals = Window.partitionBy(*group_cols, target_col)

    df_mode = (
      df
      .withColumn('counts', F.count(F.col(target_col)).over(window_grp_vals))
      .withColumn('counts_max', F.max('counts').over(window_grp))
      .filter(f'counts = counts_max')
      .dropDuplicates(subset=[*group_cols, target_col])
      .withColumn('multimodal', F.count(target_col).over(window_grp)>1)
      .withColumnRenamed(target_col, 'mode')
      .withColumnRenamed('counts_max', 'mode_counts')
      .dropDuplicates(subset=[*group_cols])
      .select(*group_cols, 'mode', 'mode_counts', 'multimodal')
    )
  return df_mode

In [82]:
calculate_mode(df_test, 'values', ['main_group','inner_group']).show()

+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+

